In [28]:
from bs4 import BeautifulSoup as BS
from urllib.parse import urlparse
from urllib.request import urlopen
import requests
import sys
import os


In [29]:
def get_links(link, query):
    all_fetched_links = []
    webpage = requests.get(f"{link}/wiki/Category:{query}")
    
    soup = BS(webpage.content, "html.parser")
    if not soup:
        return False
    else:
        # find div
        # find ul in div
        # find li in ul
        for links in soup.find_all('div', {'class': 'mw-content-ltr'}):
            for ul in links.find_all('ul'):
                for li in ul.find_all('li'):
                    for a in li.find_all('a'):
                        # if not a['href'].startswith('https://'):
                            # print(a['href'])
                        if not a['href'].startswith('https://'):
                            all_fetched_links.append(a['href'])
                        else: pass
        
        return all_fetched_links
    
def look_for_file(filename):
    fn = f"sub/{filename}.txt"
    if os.path.isfile(fn) and os.access(fn, os.R_OK):
        ask = str(input(("file is exists do you want to override? [Y/n]: ")))
        if ask in ('Y', 'y'):
            os.remove(fn)
            
        else:
            print("writing stopped")
            sys.exit(1)
    return fn

            
def write_to_file(web_source: str, q: str, data: list) -> None:
    # if file exist ask user
    # parse main url
    webpage = urlparse(web_source)
    url = f"{webpage[0]}://{webpage[1]}"    
    
    fn = look_for_file(q)
    print(fn)
    
    with open(fn, "w") as f:
        for link in data:
            f.write(f"{url}/{link}\n")
        print("done.. 👍🏻")
                
    

In [30]:
def get_text(directory: str) -> None:
    """directory is location of sub files"""
    # dir = '../sub'
    for i in os.listdir(directory):
        fn = f"subsub/{os.path.splitext(i)[0]}_info.txt"

        # dosyayı oku
        with open(os.path.join(directory, i), "r") as f:
            ret = f.readlines()
            
    #print(fn, ret)
    return fn, ret

def get_information(urls: list) -> None:  
    fn, url_list = urls

    for url in url_list:
        if 'Category' not in url:
            #fn = url.split('/')[-1]
            html = urlopen(url).read()
            soup = BS(html, features="html.parser")
            page = soup.find('p').get_text()
            #print(page)
            with open(f"{fn}", "a") as f:
                f.write(f"{page}")
    

In [31]:

q = "Turkish brands"
link = "https://en.wikipedia.org"
ret = get_links(link, q)
write_to_file(link, q, ret)
urls = get_text('sub')
get_information(urls)


sub/Turkish brands.txt
done.. 👍🏻


KeyboardInterrupt: 